In [ ]:
# import os
# import random
# from PIL import Image, ImageDraw
# from tqdm import tqdm

# # ========== CONFIG ==========
# OUTPUT_DIR = "dataset"
# IMG_DIR = os.path.join(OUTPUT_DIR, "images")
# CAPTION_FILE = os.path.join(OUTPUT_DIR, "captions.txt")

# IMAGE_SIZE = 512       # 输出图片大小
# OBJECT_SIZE = 120      # 形状大小
# NUM_IMAGES = 1000      # 生成数量，可改 2000、5000
# # ===========================

# os.makedirs(IMG_DIR, exist_ok=True)

# colors = {
#     "red":    (255, 0, 0),
#     "blue":   (0, 0, 255),
#     "green":  (0, 255, 0),
# }

# shapes = ["circle", "square"]

# relations = [
#     "to the left of",
#     "to the right of",
#     "above",
#     "below",
# ]

# def draw_shape(draw, shape, color, x, y, size):
#     if shape == "circle":
#         draw.ellipse((x, y, x+size, y+size), fill=color)
#     else:
#         draw.rectangle((x, y, x+size, y+size), fill=color)

# captions = []

# for i in tqdm(range(NUM_IMAGES), desc="Generating images"):
#     # choose attributes
#     shape1, shape2 = random.sample(shapes, 2)
#     color1, color2 = random.sample(list(colors.keys()), 2)
#     relation = random.choice(relations)

#     # base canvas
#     img = Image.new("RGB", (IMAGE_SIZE, IMAGE_SIZE), (255, 255, 255))
#     draw = ImageDraw.Draw(img)

#     # object positions
#     # start with random baseline
#     x1 = random.randint(50, 300)
#     y1 = random.randint(50, 300)

#     # determine second object’s position based on relation
#     if relation == "to the left of":
#         x2 = x1 + 160
#         y2 = y1
#     elif relation == "to the right of":
#         x2 = x1 - 160
#         y2 = y1
#     elif relation == "above":
#         x2 = x1
#         y2 = y1 + 160
#     elif relation == "below":
#         x2 = x1
#         y2 = y1 - 160

#     # draw shapes
#     draw_shape(draw, shape1, colors[color1], x1, y1, OBJECT_SIZE)
#     draw_shape(draw, shape2, colors[color2], x2, y2, OBJECT_SIZE)

#     # save image
#     filename = f"{i:05d}.png"
#     img.save(os.path.join(IMG_DIR, filename))

#     # caption
#     caption = f"a {color1} {shape1} {relation} a {color2} {shape2}"
#     captions.append(f"{filename}\t{caption}")

# # save captions
# with open(CAPTION_FILE, "w") as f:
#     f.write("\n".join(captions))

# print("Dataset generated!")


Generating images: 100%|██████████| 1000/1000 [00:01<00:00, 745.98it/s]

Dataset generated!


In [ ]:
import json
import os
import requests
from io import BytesIO
from PIL import Image
from tqdm import tqdm

jsonl_path = "all_vsr_raw_data.jsonl"
out_root = "vsr_sd_full"

os.makedirs(f"{out_root}/images", exist_ok=True)
cap_file = open(f"{out_root}/captions.txt", "w", encoding="utf-8")

idx = 0

count = 0
for line in tqdm(open(jsonl_path, "r")):
    obj = json.loads(line)

    if obj["label"] != 1:
        continue

    url = obj["image_link"]
    caption = obj["caption"]

    # download
    try:
        resp = requests.get(url, timeout=10)
        img = Image.open(BytesIO(resp.content)).convert("RGB")
    except Exception as e:
        print("Failed:", url, e)
        continue

    img = img.resize((512, 512), Image.BICUBIC)
    
    out_name = f"{idx:05d}.png"
    img.save(f"{out_root}/images/{out_name}")

    cap_file.write(f"{out_name}\t{caption}\n")
    idx += 1

cap_file.close()
print("Done! Total images:", idx)

5000it [09:39,  8.62it/s]

Done! Total images: 2482
